# Fluffy recognition model

Trying to build a model that recognises whether an image has something fluffy in it.

## Prepare environment

Install & import fastai / fastbook libraries

In [9]:
!pip install fastai==2.5
!pip install -Uqq fastbook

import fastbook
from fastbook import *
from fastai.vision.all import *

fastbook.setup_book()

Mounted at /content/gdrive


Download images

In [12]:
IMG_URL = "https://github.com/mihailthebuilder/fluffy-nb/raw/main/fluffy-images.tar.xz"
path = untar_data(IMG_URL)
get_image_files(path)

(#283) [Path('/root/.fastai/data/fluffy-images/NATURAL-GAS-VS-PROPANE.jpg'),Path('/root/.fastai/data/fluffy-images/fvfrfn6.jpg'),Path('/root/.fastai/data/fluffy-images/fluffy-cat-funny-pics-31__605.jpg'),Path('/root/.fastai/data/fluffy-images/GETTYIMAGES-976208412.jpg'),Path('/root/.fastai/data/fluffy-images/35-SEWING-NEEDLE.JPEG.jpeg.jpg'),Path('/root/.fastai/data/fluffy-images/wvjoxyi.jpg'),Path('/root/.fastai/data/fluffy-images/DOWNLOAD (8).jpg'),Path('/root/.fastai/data/fluffy-images/BOTTLES_7.jpg'),Path('/root/.fastai/data/fluffy-images/MERLIN_195320133_667F4C5E-8088-404C-ACDE-A1102C55FDB3-MOBILEMASTERAT3X.jpg'),Path('/root/.fastai/data/fluffy-images/OPERATION_UPSHOT-KNOTHOLE_-_BADGER_001.jpg')...]

## Prepare data

Prepare data for model training

In [13]:
def is_fluffy(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_fluffy, item_tfms=Resize(224))

## Train model

For training our model, we start by fetching the pre-trained ResNet model architecture with 18 layers. We then fine tune this model to our task in 2 epochs.

In [14]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(2)

epoch,train_loss,valid_loss,error_rate,time
0,1.370373,1.471668,0.232143,00:04


epoch,train_loss,valid_loss,error_rate,time
0,1.121859,0.766152,0.250000,00:04
1,0.877158,0.626377,0.232143,00:04


The error rate for the model should be somewhere around 20-25%. This was the best accurace I could achieve after...
- Increasing the number of images fed into it (currently 283).
- Tweaking the number of epochs.
- Changing the number of layers in the ResNet model architecture I was using.

## Try out model

Upload your image

In [15]:
uploader = widgets.FileUpload()
uploader

FileUpload(value={}, description='Upload')

Apply model on image

In [17]:
img = PILImage.create(uploader.data[0])
fluffy,_,probs = learn.predict(img)
print(f"Is this fluffy?: {fluffy}.")
print(f"Probability it's fluffy: {probs[1].item():.6f}")

Is this fluffy?: True.
Probability it's fluffy: 0.646041
